In [1]:
import pandas as pd
import numpy as np
import operator

In [2]:
import sys
sys.path.append('../../../src')
from preprocess_utils import interactions, correlated_cols, cols_to_drop_base_on_pval, cols_pval

In [3]:
data_dir = '../data'

In [4]:
train = pd.read_csv(f"{data_dir}/raw/train.csv", index_col='id').convert_dtypes()
display(train.shape)
train.head(2)

(300000, 15)

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331


### Interactions

In [8]:
cols = [c for c in train.columns if c != 'target']

In [9]:
_train = interactions(train[cols], [operator.truediv, operator.mul, operator.add, operator.sub], 2)
_train.shape

(300000, 378)

In [11]:
cols_to_drop = correlated_cols(_train, 0.9)

In [12]:
_train['target'] = train['target']

In [13]:
_train = _train.drop(columns=cols_to_drop)

In [14]:
from scipy.stats import pearsonr

In [15]:
len(_train.columns)

170

In [16]:
p_val_cols_todrop = cols_to_drop_base_on_pval(_train, 0.001)

In [23]:
p_val_cols = cols_pval(_train)
p_val_cols_todrop = [x[0] for x in sorted(p_val_cols.items(), key=lambda item: item[1])[30:]]

In [24]:
len(p_val_cols_todrop)

139

In [25]:
_train.drop(columns=p_val_cols_todrop).to_parquet(f"{data_dir}/interim/train_interations.parq", engine='pyarrow')

In [26]:
_train.shape

(300000, 170)

In [27]:
test = pd.read_csv(f"{data_dir}/raw/test.csv", index_col='id').convert_dtypes()
_test = interactions(test, [operator.truediv, operator.mul, operator.add, operator.sub], 2)
_test.drop(columns=cols_to_drop+p_val_cols_todrop).to_parquet(f"{data_dir}/interim/test_interations.parq", engine='pyarrow')

In [28]:
_test.shape

(200000, 378)